## Dissertation Data Cleaning and Early Modelling
### Gavin Rolls

Migrated from OvertureTest.ipynb. I've queried all of my Overture and OSM data in DataCollection.ipynb
This document is dedicated to cleaning my data and running some early iterations of employment models

### Imports

In [7]:
#Basics
import pandas as pd
import geopandas as gpd

#Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

#Shapely
from geopy.geocoders import Nominatim
from shapely.geometry import box
from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon
from shapely.geometry import mapping
from shapely.geometry import shape

#Plots!
import matplotlib.pyplot as plt

#Spatial Indexing
import h3

#Warning Supression
import warnings

## Data Loading + Cleaning

### Employment Data

In [47]:
#Skip the first six rows because they're header information
empl_data = pd.read_csv('data/employment_data/lsoa_by_industry_london.csv', skiprows=7, delimiter=',')

unnamed_cols = empl_data.columns[empl_data.columns.str.contains('^Unnamed:')]
empl_data.drop(columns=unnamed_cols, inplace=True)

#Separate name into LSOA11CD and LSOA11NM
def split_column(value):
    #Keep Greater London stats
    if value.startswith('gor:'):
        return value, value
    #Split into name and code
    else:
        parts = value.split('lsoa2011:')[1]
        code, name = parts.split(' : ')
        return code.strip(), name.strip()
        return code, name

empl_data[['LSOA11CD', 'LSOA11NM']] = empl_data['Area'].apply(lambda x: pd.Series(split_column(x)))

print("Num Rows (LSOAs) Before Cleaning: " + str(empl_data.shape[0]))

#There appear to be a bunch of duplicates so I'm going to get rid of them now
empl_data.drop_duplicates(inplace=True)

print("Num Rows (LSOAs) After Cleaning: " + str(empl_data.shape[0]))

Num Rows (LSOAs) Before Cleaning: 9478
Num Rows (LSOAs) After Cleaning: 4836


In [49]:
#Create Total Column
empl_data['total'] = empl_data[empl_data.columns[1:-2]].sum(axis=1)

empl_data.head()

,Area,"01 : Crop and animal production, hunting and related service activities",02 : Forestry and logging,03 : Fishing and aquaculture,05 : Mining of coal and lignite,06 : Extraction of crude petroleum and natural gas,07 : Mining of metal ores,08 : Other mining and quarrying,09 : Mining support service activities,10 : Manufacture of food products,...,93 : Sports activities and amusement and recreation activities,94 : Activities of membership organisations,95 : Repair of computers and personal and household goods,96 : Other personal service activities,97 : Activities of households as employers of domestic personnel,98 : Undifferentiated goods- and services-producing activities of private households for own use,99 : Activities of extraterritorial organisations and bodies,LSOA11CD,LSOA11NM,total
0,gor:London,1250,1250,400,0,1500,0,450,350,32000,...,56000,57000,17000,62000,0,0,0,gor:London,gor:London,5605320
1,lsoa2011:E01000907 : Camden 001A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,E01000907,Camden 001A,115
2,lsoa2011:E01000908 : Camden 001B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,E01000908,Camden 001B,180
3,lsoa2011:E01000909 : Camden 001C,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,E01000909,Camden 001C,200
4,lsoa2011:E01000912 : Camden 001D,0,0,0,0,0,0,0,0,0,...,0,0,0,5,0,0,0,E01000912,Camden 001D,825


### LSOA Geographic Data

In [ ]:
#Get London LSOA Shapefile Data
lsoa_geo = gpd.read_file('data/lsoa_data/s/LSOA_2011_London_gen_MHW.shp')

#Convert to WGS for consistency
lsoa_geo = lsoa_geo.to_crs(epsg=4326)

lsoa_geo = lsoa_geo.drop(lsoa_geo.columns[list(range(3, 8))], axis = 1)

print("Num Rows: " + str(lsoa_geo.shape[0]))

#Get rid of columns I'm not using for now
lsoa_geo = lsoa_geo.drop(columns=['MSOA11CD', 'HHOLDRES', 'COMESTRES', 'POPDEN', 'HHOLDS', 'AVHHOLDSZ'])
lsoa_geo = lsoa_geo.rename(columns={'USUALRES': 'population'})

lsoa_geo.head()

### Overture Places Data

### Overture Buildings Data

### OpenStreetMap Buildings Data